In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setMaster("local[1]").setAppName("ex34")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("ex34").getOrCreate()

25/01/09 15:45:28 WARN Utils: Your hostname, paloma-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/09 15:45:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/09 15:45:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [28]:
schema = "sensorId STRING, date STRING, PM10 FLOAT"
data = spark.read.csv("/home/paloma/Downloads/jupyter_spark/ex34/input.csv", header=False, schema=schema)
data.show()

+--------+----------+----+
|sensorId|      date|PM10|
+--------+----------+----+
|      s1|2016-01-01|20.5|
|      s2|2016-01-01|30.1|
|      s1|2016-01-02|60.2|
|      s2|2016-01-02|20.4|
|      s1|2016-01-03|55.5|
|      s2|2016-01-03|52.5|
+--------+----------+----+



In [30]:
data = data.rdd
# deixando a coluna de float com 1 casa decimal
pm_rounded = data.map(lambda line: (line.sensorId, line.date, round(line.PM10, 1)))
pm_rounded.collect()

[('s1', '2016-01-01', 20.5),
 ('s2', '2016-01-01', 30.1),
 ('s1', '2016-01-02', 60.2),
 ('s2', '2016-01-02', 20.4),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

In [34]:
# obtendo valores da terceira coluna (posicao 2 de cada tupla) e ao final o valor maximo
# o map vai retornar uma lista com os valores da terceira coluna e o max sera aplicado a ela
max_pm = pm_rounded.map(lambda line: line[2]).max()
print(max_pm)

60.2


In [33]:
# obter linhas que possuem o mesmo id da linha que possui o valor max de PM10
lines_max = pm_rounded.filter(lambda line: line[2] == max_pm)
lines_max.collect()

[('s1', '2016-01-02', 60.2), ('s1', '2016-01-03', 60.2)]